# The PolInSAR Course - July 01, 2024

# SAR Tomography (TomoSAR)

# Part 2 : TomoSAR reflectivity reconstruction

- P-band real TomoSAR data set: Mondah site, AfriSAR campaign
  - 11 images, flown vertical non-uniform displacements: (0, 80, 60, 40, 20, 10, -20, -40, -60, -80, -30)
  - corresponding LVIS ground topography and top canopy height provided
  - slc (hh, hv, vv), kz, flat-earth, and X-band dem are provided
- Path: '/projects/data/05-tomosar/'

Objective:

- Reconstruct (and compare) the 3D reflectivity distribution by using the Fourier and Capon algorithms.
- Display profiles in the azimuth-height plane, and for multiple range-azimuth planes at different heights.
- Replicate the results for different displacement configurations.


In [ ]:
# import useful libraries, functions, and modules
# To download the data for tomosar
from pysarpro import io, data
from pysarpro.io import rrat

#data.download_all(directory="/projects", pattern=r'^data/tomosar')

import sys
sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from tqdm import tqdm
import math as ma

%matplotlib widget

In [ ]:
def calc_tomo_fourier(a_mat,R):
    
    # Reconstructs Fourier beamforming profile from covariance natrix and steering matrix
    # Inputs :
    # a_mat         - steering matrix (heights x tracks)
    # R          - covariance matrix (tracks x tracks)

    # Outputs :
    # tomo_profile - TomoSAR profile 1D along vertical axis (a_mat.shape[0])
        
    tracks=a_mat.shape[0]
    w=a_mat/tracks
    
    tomo_profile=  np.diag ( np.linalg.multi_dot( [np.conj(w.T),R,w]))

    return tomo_profile

In [ ]:
def calc_tomo_capon(a_mat,R,diag_load):
    # Reconstructs Capon profile from covariance natrix and steering matrix
    # Inputs :
    # a_mat         - steering matrix (heights x tracks)
    # R          - covariance matrix (tracks x tracks)
    # diag_load - diagonal loading
    
    # Outputs :
    # tomo_profile - TomoSAR profile 1D along vertical axis (a_mat.shape[0])
    
    
    
    return tomo_profile

In [ ]:
# --- inputs 

# path to data
path2data = '/projects/data/tomosar/'

# Output range resolution, in meters
resrg = 20.
resaz = 20.

# pixel spacing, m
spacrg = 0.81009310
spacaz = 1.1988876

# number of tracks
tracks = 11
#improc=[1,1,0,1,0,1,0,0,1,1,0]
#tracks=np.sum(np.asarray(improc))


In [ ]:
# --- Calculate number of looks

looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1

**Step 1: load data and kz, and compensate flat-earth phase**


In [ ]:
# --- load images, compensate flat-earth, and load kz

# --- load dtm 
dtm = rrat(path2data + 'lidar_dtm.rat')

# --- load chm
chm = rrat(path2data + 'lidar_chm.rat')

# get dimensions
dim = dtm.shape

# initialize slc and kz cube
slc = np.zeros((dim[0], dim[1], tracks), 'complex64')
kz  = np.zeros((dim[0], dim[1], tracks), 'float32')

# read and prepare data
# ...
for kk in range(tracks):
    print(kk)
    kz[:,:,kk]=rrat(path2data + "kz_"+str(kk).zfill(2)+".rat" )
    slc[:,:,kk]=rrat(path2data + "slc_"+str(kk).zfill(2)+"_HV"+".rat" )
    pha_dem=rrat(path2data + "pha_dem_"+str(kk).zfill(2)+".rat" )
    slc[:,:,kk]=slc[:,:,kk]*np.exp(1j*pha_dem)
    


**Step 2: display lidar ground topography and top canopy height, and slc**


In [ ]:
# --- display

plt.figure( figsize = (9, 10))

plt.subplot(1, 3, 1)
plt.imshow(dtm, vmin = 0, vmax = 30, cmap = 'jet', aspect = 'auto')
plt.title('Lidar DTM')
plt.colorbar(orientation = 'horizontal', pad = 0.05)

plt.subplot(1, 3, 2)
plt.imshow(chm, vmin = 0, vmax = 50, cmap = 'jet', aspect = 'auto')
plt.title('Lidar CHM')
plt.colorbar(orientation = 'horizontal', pad = 0.05)

slcamp = np.sqrt( filters.uniform_filter(np.abs(slc[:, :, 5])**2, [looksa, looksr]) )

plt.subplot(1, 3, 3)
plt.imshow(slcamp, vmin = 0, vmax = 2.5*np.mean(slcamp), cmap = 'gray', aspect = 'auto')
plt.title('SLC amplitude')
plt.colorbar(orientation = 'horizontal', pad = 0.05)


In [ ]:
# --- analyse distribution of kz - just average along azimuth
plt.figure()
for ii in range(tracks):
    plt.plot(np.median(kz[:,:,ii],axis=0))


**Step 3: calculate Fourier and Capon TomoSAR profiles for every rg-az coordinate (after down-sampling) in HH**


In [ ]:
# --- downsample range and azimuth 

rgax = np.linspace(0, dim[1]-1, int(dim[1]/looksr) )
azax = np.linspace(0, dim[0]-1, int(dim[0]/looksa) )
rgax = rgax.astype(int)
azax = azax.astype(int)

# --- make meshgrids

rgm = np.outer(np.ones(azax.size, 'float32'), rgax)
azm = np.outer(azax, np.ones(rgax.size, 'float32'))
rgm = np.reshape(rgm.astype(int), rgm.size)
azm = np.reshape(azm.astype(int), azm.size)

# --- make zaxis 

zaxis = np.linspace(-20, 100, 101)

# --- initialize cubes

cubeF = np.zeros((azax.size, rgax.size, zaxis.size), 'float32')
cubeC = np.zeros((azax.size, rgax.size, zaxis.size), 'float32')

dim_tomo=cubeF.shape

# --- now start to process ...
for nn in tqdm(range(rgm.size )) :
   
    
    ii = np.unravel_index(nn,dim_tomo[0:2])
    min_az =  np.clip (azm[nn] - int(looksa/2),0,dim[0]-1)
    max_az =  np.clip (azm[nn] + int(looksa/2),0,dim[0]-1)
    min_rg =  np.clip (rgm[nn] - int(looksr/2),0,dim[1]-1)
    max_rg =  np.clip (rgm[nn] + int(looksr/2),0,dim[1]-1)   
    
    
    
    #fourier beamforming
       
    
    #R - covariance matrix
    #a_mat - steering matrix
    
    y=slc[min_az:max_az,min_rg:max_rg,:]
    R = np.einsum("arl,ark-> lk" ,  y,np.conj(y))
    
    kkz=kz[azm[nn],rgm[nn],:]
    a_mat=np.exp(-1j* np.outer(kkz,zaxis))

    cubeF[ii[0],ii[1],:]= calc_tomo_fourier(a_mat,R)

    #capon
    diag_load=10e5 
    #cubeC[ii[0],ii[1],:] =calc_tomo_capon(a_mat,R,diag_load)




    
    
    
    
    


**Step 4: compare profiles in representative transects at a fixed range coordinate**


In [ ]:
# --- downsample dtm and chm
dtm1 = dtm[azax, :]-4
dtm1 = dtm1[:, rgax]
chm1 = chm[azax, :]
chm1 = chm1[:, rgax]

# --- plot a profile for a fixed range & superimpose dtm and chm

fixrg = 500

# prepare beamforming profile
profF = np.transpose( np.squeeze(cubeF[:, int(fixrg/looksr), :]) )

# prepare capon profile
profC = np.transpose( np.squeeze(cubeC[:, int(fixrg/looksr), :]) )

# display
ii = np.where(chm1[:, int(fixrg/looksr)] > -30)
ii = ii[0]

plt.figure(figsize = (12,6))
plt.subplot(2, 1, 1)
plt.imshow(np.flipud(profF), vmin = 0, vmax = 5*np.mean(profF), aspect = 'auto', cmap = 'jet', \
           extent = [0, profF.shape[1], np.min(zaxis), np.max(zaxis)])
plt.title('Fourier')
plt.ylabel('Height (m)')
plt.xlabel('Azimuth pixel')
plt.plot(ii, dtm1[ii, int(fixrg/looksr)], '.', color = 'w')
plt.plot(ii, dtm1[ii, int(fixrg/looksr)] + chm1[ii, int(fixrg/looksr)], '.', color = 'w')
plt.axis([0, profF.shape[1], np.min(zaxis), np.max(zaxis)])

plt.subplot(2, 1, 2)
plt.imshow(np.flipud(profC), vmin = 0, vmax = 5*np.mean(profC), aspect = 'auto', cmap = 'jet', \
           extent = [0, profC.shape[1], np.min(zaxis), np.max(zaxis)]) 
plt.title('Capon')
plt.ylabel('Height (m)')
plt.xlabel('Azimuth pixel')
plt.plot(ii, dtm1[ii, int(fixrg/looksr)], '.', color = 'w')
plt.plot(ii, dtm1[ii, int(fixrg/looksr)] + chm1[ii, int(fixrg/looksr)], '.', color = 'w')
plt.axis([0, profC.shape[1], np.min(zaxis), np.max(zaxis)])

**Step 5: plot reconstructed reflectivities in rg-az at different heights**


In [ ]:
# --- find the heights above the ground

zaxis_0 = [0, 10, 20, 30, 80]

# initialize down-sampled cubes
cubeF_sel = np.zeros((azax.size, rgax.size, len(zaxis_0)), 'float32')
cubeC_sel = np.zeros((azax.size, rgax.size, len(zaxis_0)), 'float32')

# now pick the right height plane above the ground

for mm in range(len(zaxis_0))  :
    
    ix = ((dtm1 + zaxis_0[mm] - np.min(zaxis)) / (np.max(zaxis) - np.min(zaxis)) * zaxis.size)
    ix = ix.astype(int)
    
    # now move in rg-az
    for aa in range(azax.size) :
        for rr in range(rgax.size) :
            if ix[aa, rr] >= 0 and ix[aa, rr] < zaxis.size :
                cubeF_sel[aa, rr, mm] = cubeF[aa, rr, ix[aa, rr]]
                cubeC_sel[aa, rr, mm] = cubeC[aa, rr, ix[aa, rr]]

# --- display ...

# find max
maxmaxF = 0
maxmaxC = 0
for nn in range(len(zaxis_0)) :
    if np.mean(cubeF_sel[:,:,nn]) > maxmaxF : maxmaxF = np.mean(cubeF_sel[:,:,nn])
    if np.mean(cubeC_sel[:,:,nn]) > maxmaxC : maxmaxC = np.mean(cubeC_sel[:,:,nn])

# now display

plt.figure(figsize = (2*(len(zaxis_0) + 1), 10))

plt.subplot(2, (len(zaxis_0) + 1), 1)
plt.imshow(chm, vmin = 0, vmax = 50, cmap = 'jet', aspect = 'auto')
plt.title('Lidar CHM')
plt.colorbar(orientation = 'horizontal', pad = 0.05)

plt.subplot(2, (len(zaxis_0) + 1), len(zaxis_0) + 1 + 1)
plt.imshow(chm, vmin = 0, vmax = 50, cmap = 'jet', aspect = 'auto')
plt.title('Lidar CHM')
plt.colorbar(orientation = 'horizontal', pad = 0.05)

for nn in range(len(zaxis_0)) :
    
    plt.subplot(2, (len(zaxis_0) + 1), nn+1 + 1)
    plt.imshow(cubeF_sel[:,:,nn], vmin = 0, vmax = 1.5*maxmaxF, cmap = 'jet', aspect = 'auto')
    plt.title('Fou - ' + str(zaxis_0[nn]) + ' m')
    plt.colorbar(orientation = 'horizontal', pad = 0.05)
    
    plt.subplot(2, (len(zaxis_0) + 1), (len(zaxis_0) + 1) + nn + 1 +1)
    plt.imshow(cubeC_sel[:,:,nn], vmin = 0, vmax = 1.7*maxmaxC, cmap = 'jet', aspect = 'auto')
    plt.title('Cap - ' + str(zaxis_0[nn]) + ' m')
    plt.colorbar(orientation = 'horizontal', pad = 0.05)

